In [2]:
import openai
from datasets import load_dataset
from tqdm import tqdm
import random


In [3]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('open_ai')

In [4]:
# Set your OpenAI API key
openai.api_key = OPENAI_API_KEY
# Load MedQA dataset from Hugging Face
dataset = load_dataset("bigbio/med_qa", split="test[:100]")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

med_qa.py:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

bigbiohub.py:   0%|          | 0.00/19.3k [00:00<?, ?B/s]

The repository for bigbio/med_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/bigbio/med_qa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


...

/root/.cache/huggingface/datasets/downloads/extracted/454fe0fdcb22e71e2762c35bcbe1c9a23f7b0154ade8598b60ee0dc6db7d537a/data_clean/questions/US/train.jsonl


Generating test split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/454fe0fdcb22e71e2762c35bcbe1c9a23f7b0154ade8598b60ee0dc6db7d537a/data_clean/questions/US/test.jsonl


Generating validation split: 0 examples [00:00, ? examples/s]

/root/.cache/huggingface/datasets/downloads/extracted/454fe0fdcb22e71e2762c35bcbe1c9a23f7b0154ade8598b60ee0dc6db7d537a/data_clean/questions/US/dev.jsonl


In [5]:
# Convert HuggingFace format to usable dicts
qa_pairs = [{
    "question": item["question"],
    "choices": item["options"],
    "answer": item["answer"]
} for item in dataset]

# **Few Shot Prompting**

In [6]:
def get_response(messages, model="gpt-4o-mini"):
    response = openai.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

In [7]:
# Define system role
system_role = "You are a helpful and harmless AI assistant."

In [8]:
for i in range(5):
    pair = qa_pairs[i]
    print(f"Question: {pair['question']}")

    # Print choices in the desired format
    for choice in pair['choices']:
        print(f"{choice['key']} ) {choice['value']}")

    print(f"Answer: {pair['answer']}\n")

Question: A junior orthopaedic surgery resident is completing a carpal tunnel repair with the department chairman as the attending physician. During the case, the resident inadvertently cuts a flexor tendon. The tendon is repaired without complication. The attending tells the resident that the patient will do fine, and there is no need to report this minor complication that will not harm the patient, as he does not want to make the patient worry unnecessarily. He tells the resident to leave this complication out of the operative report. Which of the following is the correct next action for the resident to take?
A ) Disclose the error to the patient but leave it out of the operative report
B ) Disclose the error to the patient and put it in the operative report
C ) Tell the attending that he cannot fail to disclose this mistake
D ) Report the physician to the ethics committee
E ) Refuse to dictate the operative report
Answer: Tell the attending that he cannot fail to disclose this mista

In [9]:
user_prompt = f"""
Question: {qa_pairs[0]['question']}

Choices:
A) {qa_pairs[0]['choices'][0]['value']}
B) {qa_pairs[0]['choices'][1]['value']}
C) {qa_pairs[0]['choices'][2]['value']}
D) {qa_pairs[0]['choices'][3]['value']}
E) {qa_pairs[0]['choices'][4]['value']}

Answer: {qa_pairs[0]['answer']}  ,

Question: {qa_pairs[1]['question']}

Choices:
A) {qa_pairs[1]['choices'][0]['value']}
B) {qa_pairs[1]['choices'][1]['value']}
C) {qa_pairs[1]['choices'][2]['value']}
D) {qa_pairs[1]['choices'][3]['value']}
E) {qa_pairs[1]['choices'][4]['value']}

Answer: {qa_pairs[1]['answer']}  ,

Question: {qa_pairs[2]['question']}

Choices:
A) {qa_pairs[2]['choices'][0]['value']}
B) {qa_pairs[2]['choices'][1]['value']}
C) {qa_pairs[2]['choices'][2]['value']}
D) {qa_pairs[2]['choices'][3]['value']}
E) {qa_pairs[2]['choices'][4]['value']}

Answer: {qa_pairs[2]['answer']}  ,

Question: {qa_pairs[3]['question']}

Choices:
A) {qa_pairs[3]['choices'][0]['value']}
B) {qa_pairs[3]['choices'][1]['value']}
C) {qa_pairs[3]['choices'][2]['value']}
D) {qa_pairs[3]['choices'][3]['value']}
E) {qa_pairs[3]['choices'][4]['value']}

Answer: {qa_pairs[3]['answer']} ,

Question: {qa_pairs[4]['question']}

Choices:
A) {qa_pairs[4]['choices'][0]['value']}
B) {qa_pairs[4]['choices'][1]['value']}
C) {qa_pairs[4]['choices'][2]['value']}
D) {qa_pairs[4]['choices'][3]['value']}
E) {qa_pairs[4]['choices'][4]['value']}

Answer:

"""


In [10]:
messages=[{'role':'system','content':system_role},
          {'role':'user','content':user_prompt}
          ]

In [11]:
print(get_response(messages))

The appropriate treatment for this patient, who is experiencing itchy, watery eyes and conjunctival injection consistent with allergic conjunctivitis, would be:

**B) Ketotifen eye drops**

Ketotifen is an antihistamine that will help alleviate the allergic symptoms by blocking histamine receptors and is commonly used to treat allergic conjunctivitis.


# **One Shot Prompting**

In [12]:
user_prompt = f"""
Question: {qa_pairs[0]['question']}

Choices:
A) {qa_pairs[0]['choices'][0]['value']}
B) {qa_pairs[0]['choices'][1]['value']}
C) {qa_pairs[0]['choices'][2]['value']}
D) {qa_pairs[0]['choices'][3]['value']}
E) {qa_pairs[0]['choices'][4]['value']}

Answer: {qa_pairs[0]['answer']}  ,
Question: {qa_pairs[4]['question']}

Choices:
A) {qa_pairs[4]['choices'][0]['value']}
B) {qa_pairs[4]['choices'][1]['value']}
C) {qa_pairs[4]['choices'][2]['value']}
D) {qa_pairs[4]['choices'][3]['value']}
E) {qa_pairs[4]['choices'][4]['value']}

Answer:

"""


In [13]:
messages=[{'role':'system','content':system_role},
          {'role':'user','content':user_prompt}
          ]

In [14]:
print(get_response(messages))

The patient presents with symptoms consistent with allergic conjunctivitis, characterized by itchy, watery eyes, sneezing, and a history of similar episodes during springtime. 

The most appropriate treatment for allergic conjunctivitis, particularly when there is evidence of conjunctival injection and watery discharge, is an antihistamine. 

Among the choices provided, the best option is:

**B) Ketotifen eye drops**

Ketotifen is an antihistamine with mast cell stabilizing properties, making it effective for relieving allergic symptoms in the eyes. Other options, such as erythromycin ointment or fluorometholone eye drops, are not suitable for treating allergic conjunctivitis. Warm compresses can provide comfort but do not treat the underlying cause. Latanoprost is used for glaucoma and is not appropriate in this case.


# **Zero Shot Prompting**

In [15]:
user_prompt = f"""
Question: {qa_pairs[4]['question']}

Choices:
A) {qa_pairs[4]['choices'][0]['value']}
B) {qa_pairs[4]['choices'][1]['value']}
C) {qa_pairs[4]['choices'][2]['value']}
D) {qa_pairs[4]['choices'][3]['value']}
E) {qa_pairs[4]['choices'][4]['value']}

Answer:

"""


In [16]:
messages=[{'role':'system','content':system_role},
          {'role':'user','content':user_prompt}
          ]

In [17]:
print(get_response(messages))

The clinical scenario described is consistent with allergic conjunctivitis, given the patient's symptoms of itchy, watery eyes, sneezing, and previous similar episodes during springtime. The presence of bilateral conjunctival injection with watery discharge further supports this diagnosis. 

The most effective treatment for allergic conjunctivitis typically includes antihistamines. Among the choices provided, **B) Ketotifen eye drops** are a combination of an antihistamine and a mast cell stabilizer, making them a suitable option for treating allergic conjunctivitis.

Therefore, the most appropriate treatment for this patient is:

**B) Ketotifen eye drops.**
